In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install packages

!pip install matplotlib ultralytics fastapi uvicorn boto3 pyngrok pillow python-multipart nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00


In [3]:
# import and pathes
from decimal import Decimal
import pickle
import glob
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel
import boto3
import requests
from io import BytesIO
from PIL import Image
import json
import os
from pyngrok import ngrok
import uvicorn
import threading
import asyncio
import shutil
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor
from google.colab import userdata


source_path_1 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/models/autoencoder_model.pth'
source_path_2 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/models/clustering_model.pkl'
source_path_3 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/modules/fruit_reconstructor.py'
source_path_5 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/modules/autoencoder_module.py'
source_path_4 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/models/best_2.pt'
source_path_6 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/modules/clustering_module.py'
source_path_7 = '/content/drive/MyDrive/Fruit_Recognition_Serivce/modules/image_processing_utils.py'
destination_path_1 = '/content/autoencoder_model.pth'
destination_path_2 = '/content/clustering_model.pkl'
destination_path_3 = '/content/fruit_reconstructor.py'
destination_path_4 = '/content/best_2.pt'
destination_path_5 = '/content/autoencoder_module.py'
destination_path_6 = '/content/clustering_module.py'
destination_path_7 = '/content/image_processing_utils.py'

shutil.copy(source_path_1, destination_path_1)
shutil.copy(source_path_2, destination_path_2)
shutil.copy(source_path_3, destination_path_3)
shutil.copy(source_path_4, destination_path_4)
shutil.copy(source_path_5, destination_path_5)
shutil.copy(source_path_6, destination_path_6)
shutil.copy(source_path_7, destination_path_7)
clustering_model_path = destination_path_2
autoencoder_model_path = destination_path_1
img_files = ""
yolo_model_path = r"/content/best_2.pt"

from fruit_reconstructor import train_and_save_models, load_models, infer, visualize_errors

In [ ]:
def extract_box(image, x1: int, y1: int, x2: int, y2: int): # -> ndarray:
    """
    주어진 좌표를 사용하여 이미지의 일부분을 추출합니다.

    Parameters:
    image (np.ndarray): 전체 이미지 배열
    x1, y1 (int): 좌상단 모서리 좌표
    x2, y2 (int): 우하단 모서리 좌표

    Returns:
    np.ndarray: 추출된 이미지 배열
    """
    # 이미지 배열에서 네모 박스 안의 부분을 추출
    cropped_image = image[y1:y2, x1:x2]

    return cropped_image

def infer_img(img, model):
    """
    이미지, 모델 경로를 전달하면 객체를 검출해 bbox내 이미지 배열과 mask를 반환하는 함수

    Parameters
      img
      model

    Returns
      bbox_crop(ndarray) : 내 이미지 배열 크롭
      mask(ndarray) : 마스크 배열
      lable : 라벨
    """

    result = model.predict(img, save=False)[0]  # save = True - runs/segment/predict 하위에 추론 결과 이미지 저장
    label = result.probs
    # 바운딩박스 내의 이미지 크롭해서 리턴
    image= result.orig_img  # 원본 이미지(ndarray)
    xyxy = result.boxes.xyxy[0] # 바운딩 박스 좌표(좌상단xy, 우하단 xy)
    x1, y1, x2, y2 = [int(a) for a in xyxy]
    bbox_crop =extract_box(image, x1, y1, x2, y2)[:,:,::-1]

    # 바이너리 마스크 리턴
    # 바이너리 마스크 생성
    mask = result.masks.data[0]  # GPU 텐서
    mask_cpu = mask.cpu().numpy()  # CPU로 이동 후 NumPy 배열로 변환

    # mask = np.array(result.masks.data[0])
    mask_crop = extract_box(mask_cpu, x1, y1, x2, y2)

    return bbox_crop, mask, label

In [5]:
# 모델 로드
yolo_model = YOLO(yolo_model_path)
clustering_model, autoencoder_model = load_models(clustering_path=clustering_model_path, autoencoder_path=autoencoder_model_path)

nest_asyncio.apply()

# FastAPI 앱 생성
app = FastAPI()

# parmameters
clustering_params = {
    'color_space': 'LAB',

    'use_grid_mask': True
}
autoencoder_params = {
    'color_space': 'RGB',
    'use_grid_mask': True,

}

aws_key = userdata.get('AWS_ACCESS_KEY_ID')
secret_key = userdata.get('AWS_SECRET_ACCESS_KEY')
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

# AWS S3 설정
s3_client = boto3.client('s3',
                         aws_access_key_id=aws_key,
                         aws_secret_access_key=secret_key,
                         region_name='ap-southeast-2b')

S3_BUCKET_NAME = 'itsfru-first'
RESULT_FOLDER = 'fruit_recognition_result'

# 요청 바디 모델
class ImageRequest(BaseModel):
    image_url: str

# 분석 모델
class AnalysisResult(BaseModel):
    fruit_type: str
    quality: float
    freshness: float
    discount_rate: Decimal
    result_path: str

# 과일 분석 함수
def analyze_image(img_files):
    # get box and mask
    bbox_crops, masks, lables =infer_img(img_files, yolo_model)
    quality = 0.0
    discount_rate = 0.0
    freshness = 0.0

    # 이미지 분석 로직
    recon_error, grid_cluster, grid_error, error_ratios = infer(bbox_crops, masks, clustering_model, autoencoder_model,
                                                                  clustering_params, autoencoder_params)
    print("추론 완료")
    print("Error Ratios:")
    print(f"Total Error Grids Ratio: {error_ratios[0]:.4f}")
    print(f"Cluster Error Grids Ratio: {error_ratios[1]:.4f}")
    print(f"Reconstruction Error Grids Ratio: {error_ratios[2]:.4f}")

    # 분석 값 결정 로직


    # 결과 반환
    return {
        "fruit_type": lables,
         "quality": float(quality),
        "freshness": float(freshness),
        "discount_rate": Decimal(str(round(discount_rate, 2)))
    }

@app.post("/analyze", response_model=AnalysisResult)
async def analyze_fruit(request: ImageRequest):
    try:
        # 이미지 다운로드
        response = requests.get(request.image_url)
        if response.status_code != 200:
            raise HTTPException(status_code=400, detail="Failed to download image")

        image = Image.open(BytesIO(response.content))

        # 이미지 분석
        analysis_result = analyze_image(image)

        # 결과 JSON 생성 및 S3 업로드
        result_json = json.dumps(analysis_result, default=str)  # Decimal을 문자열로 변환
        result_key = f"fruit_recognition_result/result_{os.path.basename(request.image_url)}.json"
        s3_client.put_object(Bucket=S3_BUCKET_NAME, Key=result_key, Body=result_json)

        # 결과 URL 생성
        result_url = s3_client.generate_presigned_url('get_object',
                                                      Params={'Bucket': S3_BUCKET_NAME, 'Key': result_key},
                                                      ExpiresIn=3600)
        analysis_result['result_path'] = result_url

        # AnalysisResult 형식으로 직접 반환
        return AnalysisResult(**analysis_result)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

async def run_server():
    config = uvicorn.Config(app, port=8000, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

async def setup_and_run():
    # ngrok 인증 토큰 설정
    ngrok.set_auth_token(ngrok_token)

    # ngrok으로 public URL 생성
    public_url = ngrok.connect(8000).public_url
    print(f'Ngrok URL: {public_url}')
    print(f'Full analyze URL: {public_url}/analyze')

    await run_server()

def run_fastapi():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(setup_and_run())

In [ ]:
# 서버켜고 url 확인후 환경변수에 적용
run_fastapi()

Ngrok URL: https://cf8d-34-83-45-201.ngrok-free.app
Full analyze URL: https://cf8d-34-83-45-201.ngrok-free.app/analyze


INFO:     Started server process [558]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
